In [6]:
from datasets import load_dataset,Audio
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from evaluate import load
def evaluateModel(path):
    common_voice = load_dataset("mozilla-foundation/common_voice_11_0", "zh-HK", split="test")
    common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
    common_voice = common_voice.select(range(1000))

    processor = WhisperProcessor.from_pretrained(path)
    model = WhisperForConditionalGeneration.from_pretrained(path).to("cuda")



    def map_to_pred(batch):
        audio = batch["audio"]
        input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
        batch["reference"] = processor.tokenizer._normalize(batch['sentence'])

        with torch.no_grad():
            predicted_ids = model.generate(input_features.to("cuda"))[0]
        transcription = processor.decode(predicted_ids)
        batch["prediction"] = processor.tokenizer._normalize(transcription)
        return batch

    result = common_voice.map(map_to_pred)

    wer = load("wer")
    print(100 * wer.compute(references=result["reference"], predictions=result["prediction"]))


In [7]:
whisper_path=["openai/whisper-tiny","openai/whisper-base","openai/whisper-small","openai/whisper-medium","openai/whisper-large","openai/whisper-large-v2","openai/whisper-large-v3"]

In [8]:
evaluateModel(whisper_path[0])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

2696.6162065894923


In [9]:
evaluateModel(whisper_path[1])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

2438.4683882457703


In [10]:
evaluateModel(whisper_path[2])

C:\Users\ken20\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ken20\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

1121.193232413179


In [11]:
evaluateModel(whisper_path[3])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

835.6188780053427


In [12]:
evaluateModel(whisper_path[4])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

963.3125556544968


In [13]:
evaluateModel(whisper_path[5])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

733.3036509349956


In [14]:
evaluateModel(whisper_path[6])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

105.87711487088156
